In [1]:
import pandas as pd
import numpy as np
import pymannkendall as mk
import geopandas as gpd
import plotly.express as px

st = pd.read_csv(
    filepath_or_buffer = '../Data/States.csv',
    index_col = [0],
    header = [0, 1]
)
rs = pd.read_csv(
    filepath_or_buffer = '../Data/RobustStates.csv',
    index_col = [0],
    header = [0, 1]
)
ds = pd.read_csv(
    filepath_or_buffer = '../Data/Districts.csv',
    index_col = [0],
    header = [0, 1]
)
rd = pd.read_csv(
    filepath_or_buffer = '../Data/RobustDistricts.csv',
    index_col = [0],
    header = [0, 1]
)
jss = pd.read_pickle('../Data/States.pkl')
jsd = pd.read_pickle('../Data/Districts.pkl')

def trend(x):
    if str(x) == 'nan':
        return 'No Data'
    elif x == 0:
        return 'Constant'
    elif x < 0:
        return 'Increase'
    elif x > 0:
        return 'Decrease'
    else:
        print('Problem')

def avg(x):
    z = 0
    for i in [j for j in x]:
        if str(i) == 'nan':
            z += 1
    if z == len(x):
        return np.nan
    return np.nanmean(x)

def mkd(s, p = 0):
    try:
        x = mk.original_test(s)[p]
        if x == 'decreasing':
            return 'Increase'
        elif x == 'increasing':
            return 'Decrease'
        else:
            return x
    except ZeroDivisionError:
        if p == 0:
            return 'No Data'
        return 0

def NoData(dafr, level):
    na = pd.read_pickle(f'../Data/NoData{level}s.pkl')
    nd = pd.DataFrame(
        data = {
            'Pre' : ['No Data' for i in na],
            'Post' : ['No Data' for i in na]
        },
        index = na)
    return dafr.append(nd)

def final_trend(kind, level, year = 2017):
    if level == 'State':
        df = st.copy()
    else:
        df = ds.copy()
    
    df = df[[str(i) for i in range(year - 10, year + 1)]]
    yty = df[str(year)] - df[str(year - 1)]
    yty = yty.applymap(trend)
    
    if (kind=="YTY"):
        yty = NoData(dafr = yty, level = level)
        return yty
    elif (kind == "DA"):
        if level == 'State':
            df = rs.copy()
        else:
            df = rd.copy()
        da = yty.copy()
        i = df[[str(year - 10 + i) for i in range(10)]]
        da['Pre'] =  i.iloc[:,::2].apply(avg, axis = 1)
        da['Post'] = i.iloc[:,1::2].apply(avg, axis = 1)
        da = df[str(year)] - da
        da = da.applymap(trend)
        da = NoData(dafr = da, level = level)
        return da
    
    elif(kind == "MKT"):
        mkt = yty.copy()
        i = df[[str(year - 10 + i) for i in range(10)]]
        mkt['Pre'] = i.iloc[:,::2].apply(mkd, axis = 1)
        mkt['Post'] = i.iloc[:,1::2].apply(mkd, axis = 1)
        mkt = NoData(dafr = mkt, level = level)
        return mkt
    
    else:
        print("Unavailable")

def plot(wise, map_df, monsoon):
    if wise == 'District':
        js = jsd
    else:
        js = jss
    fig = px.choropleth(map_df.reset_index(),
                        geojson = js,
                        locations = "index", 
                        featureidkey = f"properties.{wise}",
                        color = monsoon, 
                        color_discrete_map = {
                            'Increase' : '#aec6cf',
                            'Decrease' : '#ff6961',
                            'No Data' : '#666699',
                            'Constant' : '#228B22',
                            'no trend' : '#ffff4d'
                        })
    fig = fig.update_geos(
        fitbounds = "locations", 
        visible = False)
    fig.write_html('vastu.html')

def final_plot(kind, level, monsoon, year = 2017):
    map_df = final_trend(kind = kind, year = year, level = level)
    plot(wise = level, map_df = map_df, monsoon = monsoon)

In [2]:
final_plot(
    kind = 'DA',
    level = 'State',
    year = 2011,
    monsoon = 'Post'
)

KeyError: "['2001' '2002' '2003' '2004'] not in index"

In [4]:
final_trend(kind = 'DA',
    level = 'State',
    year = 2015,)

Monsoon,Pre,Post
Andhra Pradesh,Decrease,Decrease
Arunachal Pradesh,Decrease,Increase
Assam,Decrease,Increase
Bihar,Decrease,Decrease
Chandigarh,Decrease,Decrease
Chhattisgarh,Increase,Decrease
Dadra And Nagar Haveli,Decrease,Increase
Daman And Diu,Decrease,Increase
Goa,Increase,Increase
Gujarat,Decrease,Decrease
